# 1. feladat [8p]

 Alex és Bob és Charlie közös programot szerveznek. A szabad időintervallumaik az intervals szótárban vannak megadva. Készítsünk programot, amely kiszámítja, hogy hány olyan nap van, amikor mindhárman ráérnekl Az intervallumok zártak, azaz a végpontok szabad időpontnak tekintendők. A program ne csak a megadott intervals adatszerkezetre működjön, hanem tetszőleges, ugyanilyen formátumú bemenetre is! 

In [33]:
import datetime as dt

intervals = {
    'Alex': [('2020-06-20','2020-06-23'), ('2020-06-30', '2020-07-05')],
    'Bob' : [('2020-06-18','2020-06-21'), ('2020-06-24','2020-07-01'), ('2020-07-03', '2020-07-04')],
    'Charlie': [('2020-06-21', '2020-06-28'), ('2020-07-02', '2020-07-10')],
}

def get_free_dates(name, intervals):
    free_dates = []
  
    free_periods = intervals[name]

    for period in free_periods:
       start_date = dt.datetime.strptime(period[0], "%Y-%m-%d").date()
       end_date = dt.datetime.strptime(period[1], "%Y-%m-%d").date()

       free_dates_period = [start_date + dt.timedelta(days=current_day) for current_day in range((end_date - start_date).days + 1)]
       free_dates.extend([date for date in free_dates_period])

    return free_dates

def main():
    names = intervals.keys()

    req_occurs = len(names)
    
    free_dates_nested = [get_free_dates(name, intervals) for name in names]
    free_dates = [date for free_date_for_name in free_dates_nested for date in free_date_for_name]
    
    correct_dates = {date for date in free_dates if free_dates.count(date) == req_occurs}
    print(len(correct_dates))
    
main()

3


# 2. feladat [12p] 

A china.txt Kína mezőgazdaságáról tartalmaz összesitő adatokat, az 1949 és 2008 közötti időszakból. Készitsünk programot, amely beolvassa a szövegfájl tartalmát, majd válaszol az alábbi kérdésekre! 
- Összesen hány tonna trágyát használtak a 60-as években? (Áru: Total fertilizer consumption ) 
- Melyik évben volt a legmagasabb a mezőgazdaságban dolgozók száma? (Áru: Ag employment (primary industry) ) 
- A rizstermeles (Kategória: Crop production , Áru: Rice ) melyik évben hányszorosa volt az 1949-es értéknek? 
- 2007-ben melyik 5 terménynek volt átlagosan a legnagyobb a hektáronkénti hozama? (A termelési mennyiségek a crop production , a vetési területek a crop sown area kategóriában van megadva.) 


In [137]:
import pandas as pd

df = pd.read_csv("china.txt", delimiter="\t", decimal=",", skiprows=1)

# 1. feladat
fertilizer_usage = df.loc[(df["Ev"].between(1960, 1970, inclusive="left")) & (df["Aru"] == "Total fertilizer consumption")]
fertilizer_tons = fertilizer_usage["Mennyiseg"].sum() * 1000
print(f"1. feladat: {fertilizer_tons} t\n")

# 2. feladat
max_ag_employment = df.loc[df["Aru"] == "Ag employment (primary industry)"].max()
print(f"2. feladat: {max_ag_employment['Ev']}\n")

# 3. feladat
rice_prod = df.loc[(df["Kategoria"] == "Crop production") & (df["Aru"] == "Rice")].copy()
rice_prod_base_quantity = rice_prod.loc[df["Ev"] == 1949, "Mennyiseg"].item()

rice_prod["1949-hez kepest"] = rice_prod["Mennyiseg"].divide(rice_prod_base_quantity)
rice_prod_diff = rice_prod[['Ev', '1949-hez kepest']]
print(f"3. feladat:\n{rice_prod_diff}\n")

# 4. feladat
crop_prod_2007 = df.loc[(df["Ev"] == 2007) & (df["Kategoria"] == "Crop production")][["Aru", "Mennyiseg"]].copy()
crop_prod_2007 = crop_prod_2007.rename(columns={"Mennyiseg": "1000 t"})

crop_sown_area_2007 = df.loc[(df["Ev"] == 2007) & (df["Kategoria"] == "Crop sown area")][["Aru", "Mennyiseg"]].copy()
crop_sown_area_2007 = crop_sown_area_2007.rename(columns={"Mennyiseg": "1000 ha"})

crop_data_2007 = pd.merge(crop_prod_2007, crop_sown_area_2007, on="Aru")
crop_data_2007["Atlag hozam (t / ha)"] = crop_data_2007["1000 t"].divide(crop_data_2007["1000 ha"]).multiply(1000)

top_5_crops = crop_data_2007.sort_values(by="Atlag hozam (t / ha)", ascending=False)[:5]
print(f"4. feladat:\n {top_5_crops}")


1. feladat: 16057000.0 t

2. feladat: 2008

3. feladat:
        Ev  1949-hez kepest
17    2008         3.944907
173   2007         3.824319
411   2006         3.753147
655   2005         3.712365
898   2004         3.681521
1142  2003         3.302621
1388  2002         3.588015
1637  2001         3.650529
1868  2000         3.862843
2108  1999         4.080317
2338  1998         4.084962
2567  1997         4.126529
2794  1996         4.010751
3020  1995         3.807709
3244  1994         3.616672
3460  1993         3.649173
3671  1992         3.828184
3880  1991         3.778662
4087  1990         3.892096
4295  1989         3.702950
4498  1988         3.476349
4698  1987         3.582321
4897  1986         3.540426
5095  1985         3.465289
5298  1984         3.664405
5480  1983         3.471374
5660  1982         3.321924
5839  1981         2.959297
6016  1980         2.876143
6185  1979         2.955083
6339  1978         2.814883
6485  1977         2.642923
6591  1976         2